# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from mykeys import geoapi_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.81,88,95,8.19,PN,1731170933
1,1,jinghong,21.9910,100.7341,18.28,92,77,1.07,CN,1731170935
2,2,taulaga,-11.0553,-171.0883,27.93,71,33,0.61,AS,1731170936
3,3,bethel,41.3712,-73.4140,10.41,38,0,4.12,US,1731170937
4,4,gaspe,48.8334,-64.4819,2.12,55,75,8.23,CA,1731170938


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0),:
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,idri,27.5000,13.2667,21.03,34,0,2.85,LY,1731170958
79,79,cabo san lucas,22.8909,-109.9124,26.95,52,0,2.06,MX,1731170759
92,92,darab,28.7519,54.5444,21.16,22,0,1.41,IR,1731171034
153,153,kailua-kona,19.6406,-155.9956,21.98,80,0,0.00,US,1731171103
159,159,pisco,-13.7000,-76.2167,21.12,73,0,3.60,PE,1731171109
203,203,mansa,29.9833,75.3833,23.50,41,0,2.88,IN,1731171160
227,227,piton saint-leu,-21.2196,55.3151,24.06,73,0,1.03,RE,1731171185
229,229,kuchera,26.9839,73.9719,25.91,25,0,2.51,IN,1731171188
323,323,tamanrasset,22.7850,5.5228,23.95,15,0,3.09,DZ,1731171292
352,352,agadez,19.7500,10.2500,22.63,21,0,3.81,NE,1731171323


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
 
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
22,idri,LY,27.5000,13.2667,34,
79,cabo san lucas,MX,22.8909,-109.9124,52,
92,darab,IR,28.7519,54.5444,22,
153,kailua-kona,US,19.6406,-155.9956,80,
159,pisco,PE,-13.7000,-76.2167,73,
203,mansa,IN,29.9833,75.3833,41,
227,piton saint-leu,RE,-21.2196,55.3151,73,
229,kuchera,IN,26.9839,73.9719,25,
323,tamanrasset,DZ,22.7850,5.5228,15,
352,agadez,NE,19.7500,10.2500,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapi_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    print(name_address.status_code)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
200
idri - nearest hotel: No hotel found
200
cabo san lucas - nearest hotel: Comfort Rooms
200
darab - nearest hotel: محدوده هتل جهانگردی
200
kailua-kona - nearest hotel: Kona Seaside Hotel
200
pisco - nearest hotel: La Portada
200
mansa - nearest hotel: No hotel found
200
piton saint-leu - nearest hotel: Appartement Piton Saint Leu
200
kuchera - nearest hotel: No hotel found
200
tamanrasset - nearest hotel: فندق الأمان
200
agadez - nearest hotel: No hotel found
200
ar ruways - nearest hotel: No hotel found
200
warud - nearest hotel: HREEKRUPA Restaurant, LODGING & BANQUET HALL 
200
los llanos de aridane - nearest hotel: Valle Aridane


,City,Country,Lat,Lng,Humidity,Hotel Name
22,idri,LY,27.5000,13.2667,34,No hotel found
79,cabo san lucas,MX,22.8909,-109.9124,52,Comfort Rooms
92,darab,IR,28.7519,54.5444,22,محدوده هتل جهانگردی
153,kailua-kona,US,19.6406,-155.9956,80,Kona Seaside Hotel
159,pisco,PE,-13.7000,-76.2167,73,La Portada
203,mansa,IN,29.9833,75.3833,41,No hotel found
227,piton saint-leu,RE,-21.2196,55.3151,73,Appartement Piton Saint Leu
229,kuchera,IN,26.9839,73.9719,25,No hotel found
323,tamanrasset,DZ,22.7850,5.5228,15,فندق الأمان
352,agadez,NE,19.7500,10.2500,21,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)